## Cheat Sheep for Deep Learning Models

### 1 SoftMax回归
#### 1.1 模型定义
如无特别说明，向量均指列向量。

softmax回归是logistic回归（适用于2类分类）扩展到多类分类的结果。设标签$c \in \{1, ..., C \}$，对于样本$(\vec{x}, y)$，
softmax回归预测样本标签为$c$的概率为
$$
p(y=c|\vec{x}) = softmax(\vec{w}_c^\top \vec{x}) = \frac{\exp (\vec{w}_c^\top \vec{x})}{\sum_{c'=1}^C \exp (\vec{w}_{c'}^\top \vec{x})},
$$
因此softmax回归的预测结果为
$$
\hat{y} = argmax_{c=1}^C p(y=c|\vec{x}) = argmax_{c=1}^C \vec{w}_c^\top \vec{x}.
$$
本质上，softmax回归是一个单层神经网络，输出层为softmax层。

#### 1.2 单样本的矢量计算表达式
为了更方便地定义torch tensor，假设所有向量均为**行向量**。

设$d$为样本特征个数且$\vec{x} \in \mathbb{R}^{1 \times d}$，$W \in \mathbb{R}^{d \times C}$为待学习的权重，$\vec{b} \in \mathbb{R}^{1 \times C}$为偏置，则对于样本$(\vec{x}^{(i)}, y^{(i)})$，softmax回归的矢量计算表达式为
$$
\hat{\vec{y}}^{(i)} = softmax (\vec{x}^{(i)} W + \vec{b}),
$$
其中$\hat{\vec{y}}^{(i)} \in \mathbb{R}^{C}$的各个元素反应了softmax回归预测各标签的概率。

#### 1.3 多样本的矢量计算表达式
为了更方便地定义torch tensor，假设所有向量均为行向量。

令$X \in \mathbb{R}^{n \times d}$是$n$个样本的特征矩阵，则
$$
\hat{Y} = softmax (XW+b).
$$
PyTorch会自动开启广播模式。

#### 1.4 参数学习
采用交叉熵损失函数，只关心正确类别的预测概率：
$$
l (W, \vec{b}) = - \frac{1}{N} \sum_{n=1}^N \sum_{c=1}^C y_c^{(n)} \log \hat{y}_c^{(n)}.
$$
其梯度的求导过程并不难，虽然略复杂。最终参数的更新公式为
$$
W_{t+1} \leftarrow W_t + \alpha \bigg( \frac{1}{|\mathcal{B}|} \sum_{n \in \mathcal{B}} \vec{x}^{(n)} \Big(\vec{y}^{(n)} - \hat{\vec{y}}^{(n)}_{W_t}\Big)^\top \bigg)
$$
$$
b_{t+1} \leftarrow b_t + \alpha \bigg( \frac{1}{|\mathcal{B}|} \sum_{n \in \mathcal{B}} \Big(\vec{y}^{(n)} - \hat{\vec{y}}^{(n)}_{b_t}\Big) \bigg)
$$
其中$\vec{y}^{(n)}$是一个one-hot向量，仅有true label位置对应的元素为1。

### 2 多层感知器（MLP）
#### 2.1 隐藏层
多层感知机（Multi-layer Perceptron）在单层神经网络的基础上引入了一到多个隐藏层（hidden layer）。隐藏层位于输入层和输出层之间。下图展示了一个多层感知机的神经网络图，它含有一个隐藏层，该层中有5个隐藏单元。多层感知机中的隐藏层和输出层都是全连接层。

<img src='../resource/mlp.svg'>

#### 2.2 矢量计算表达式
在不考虑激活函数的前提下，设输入样本$X \in \mathbb{R}^{n \times d}$，标签个数为$q$。对于仅包含单个隐藏层的神经网络，记隐藏层的输出为$H \in \mathbb{R}^{n \times h}$。则
$$
H = X W_h + b_h
$$
$$
O = H W_o + b_o,
$$
其中$W_h \in \mathbb{R}^{d \times h}$，$b_h \in \mathbb{R}^{1 \times h}$，$W_o \in \mathbb{R}^{h \times q}$，$b_o \in \mathbb{R}^{1 \times q}$分别为隐藏层和输出层的权重及偏置。

因此
$$
O = X W_h W_o + (b_h W_o + b_o),
$$
这相当于是一个权重为$W_h W_o$，偏置为$b_h W_o + b_o$的单层神经网络。

#### 2.3 激活函数
全连接层只是对数据做仿射变换（affine transformation），而**多个仿射变换的复合仍然是一个仿射变换**。解决问题的一个方法是引入非线性变换，例如对隐藏变量使用按元素运算的非线性函数进行变换，然后再作为下一个全连接层的输入。这个非线性函数被称为激活函数（activation function）。

* ReLU（Rectified Linear Unit）
$$
ReLU(x) = \max (x, 0)
$$
* sigmoid
$$
sigmoid (x) = \frac{1}{1 + \exp (-x)}
$$
* tanh
$$
tanh (x) = \frac{1 - \exp (-2x)}{1 + \exp (-2x)}
$$

#### 2.4 多层感知机
多层感知机就是含有至少一个隐藏层的由全连接层组成的神经网络，且每个隐藏层的输出通过激活函数进行变换。多层感知机的层数和各隐藏层中隐藏单元个数都是超参数。以单隐藏层为例，
$$
H = \phi (X W_h + b_h)
$$
$$
O = H W_o + b_o
$$
在分类问题中，我们可以对输出$O$做softmax运算，并使用softmax回归中的交叉熵损失函数。在回归问题中，我们将输出层的输出个数设为$1$，并将输出$O$直接提供给线性回归中使用的平方损失函数。

### 3 权重衰减（weight decay）
应对过拟合的方法是**正则化**。以线性回归问题为例，默认的均方误差为
$$
l(w_1, w_2, b) = \frac{1}{n} \sum_{i=1}^n \frac{1}{2} \bigg( x_1^{(i)}w_1 + x_2^{(i)}w_2 + b - y^{(i)} \bigg)^2,
$$
若加上$L_2$范数惩罚项，则得到新损失函数：
$$
l(w_1, w_2, b) + \frac{\lambda}{2n} \Vert \vec{w} \Vert^2.
$$
很容易计算，若对新损失函数求$\vec{w}$的偏导，则可以得到$\vec{w}$的更新公式为
$$
w_1 \leftarrow \bigg( 1 - \frac{\eta \lambda}{|\mathcal{B}|} \bigg) w_1 - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} x_1^{(i)} \bigg( x_1^{(i)}w_1 + x_2^{(i)}w_2 + b - y^{(i)} \bigg)\\
w_2 \leftarrow \bigg( 1 - \frac{\eta \lambda}{|\mathcal{B}|} \bigg) w_2 - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} x_2^{(i)} \bigg( x_1^{(i)}w_1 + x_2^{(i)}w_2 + b - y^{(i)} \bigg),
$$
这相当于是令权重先自乘小于$1$的数，再减去不含惩罚项的梯度。因此，$L_2$范数正则化又叫权重衰减。

### 4 Dropout
在讲解MLP时我们给出了如下图所示的带有隐藏层的神经网络：
<img src='../resource/mlp.svg'>
其中，对于单个样本$\big([x_1, ..., x_4]^\top, y\big)$，隐藏单元$h_i$的计算表达式为
$$
h_i = \phi \Big(\vec{x}^\top W_h(:,i) + b_h(i)\Big)
$$

若对该隐藏层使用dropout，则该层的每个隐藏单元有一定概率会被丢弃掉。设丢弃概率（超参数）为$p$，则$\forall i, h_i$有$p$的概率会被清零，有$1-p$的概率会被做拉伸。用数学语言描述即
$$
h'_i = \frac{\xi_i}{1-p}h_i,
$$
其中$\xi_i$是一个随机变量，$p(\xi_i = 0) = p$，$p(\xi_i=1) = 1-p$。
则
$$
\mathbb{E} [h'_i] = h_i.
$$
这意味着**dropout不改变输入的期望输出**（这就是要除以$1-p$的原因）。

对上述MLP训练的时候使用dropout，一种可能的网络结构如下：
<img src='../resource/dropout.svg'>
此时MLP的输出不依赖$h_2$和$h_5$。由于在训练中隐藏层神经元的丢弃是随机的，即$h_1, ..., h_5$都有可能被清零，输出层的计算无法过度依赖$h_1, ..., h_5$中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。

**Dropout是一种训练时应对过拟合的方法，并未改变网络的结构。当参数训练完毕并用于测试时，任何参数都不会被dropout。**

### 5 反向传播的数学原理
到目前为止，我们只定义了模型的正向传播（forward）的过程，梯度的反向传播则是PyTorch自动实现的。接下来将以带$L_2$范数正则化项的、包含单个隐藏层的MLP解释反向传播的数学原理。

#### 5.1 正向传播
不考虑偏置，设输入$\vec{x} \in \mathbb{R}^d$，则得到中间变量$\vec{z} = W^{(1)} \vec{x} \in \mathbb{R}^h$，其中$W^{(1)} \in \mathbb{R}^{h \times d}$为隐藏层的权重，其中$h$是隐藏层神经元的个数；

$\vec{z}$作为输入传递给激活函数$\phi$，得到$\vec{h} = \phi(\vec{z}) \in \mathbb{R}^h$；

将$\vec{h}$传递给输出层，得到$\vec{o} = W^{(2)} \vec{h} \in \mathbb{R}^q$，其中$W^{(2)} \in \mathbb{R}^{q \times h}$为输出层的权重，$q$为输出层神经元的个数（即label的个数）。

设损失函数为$l$，且样本标签为$y$，则单个样本的loss为$L = l(\vec{o},y)$。考虑$L_2$正则化项$s = \frac{\lambda}{2} \bigg( \Vert W^{(1)} \Vert^2_F + \Vert W^{(2)} \Vert^2_F \bigg)$，则单个样本上的优化目标为
$$
J = L + s = l(\vec{o},y) + \frac{\lambda}{2} \bigg( \Vert W^{(1)} \Vert^2_F + \Vert W^{(2)} \Vert^2_F \bigg).
$$
正向传播的计算图如下：
<img src='../resource/forward.svg'>

#### 5.2 反向传播
反向传播依据微积分中的链式法则，沿着从输出层到输入层的顺序，依次计算并存储目标函数有关神经网络各层的中间变量以及参数的梯度。
第$l$层的误差可由第$l+1$层的误差得到。

**张量求导的链式法则**:

对于任意形状的张量$X,Y,Z$，若$Y = f(X), Z = f(Y)$，则
$$
\frac{\partial Z}{\partial X} = prod(\frac{\partial Z}{\partial Y}, \frac{\partial Y}{\partial X}),
$$
其中$prod(\cdot)$运算符将根据两个输入的形状，在**必要的操作（如转置和互换输入位置）后**对两个输入做乘法。

**计算$\frac{\partial J}{\partial W^{(2)}}$**:

将应用链式法则依次计算各中间变量和参数的梯度，其计算次序与前向传播中相应中间变量的计算次序恰恰相反。

首先$J = L + s$（简单起见，仅考虑单个样本），所以<font color='blue'>$\frac{\partial J}{\partial L} = 1, \frac{\partial J}{\partial s} = 1$</font>；

其次，由于$L = l(\vec{o}, y)$，所以<font color='blue'>$\frac{\partial J}{\partial \vec{o}} = prod(\frac{\partial J}{\partial L}, \frac{\partial L}{\partial \vec{o}}) = \frac{\partial L}{\partial \vec{o}}$</font>（这与损失函数$l$的选取有关，因此不再继续展开）；

因为$s = \frac{\lambda}{2} \bigg( \Vert W^{(1)} \Vert^2_F + \Vert W^{(2)} \Vert^2_F \bigg)$，所以$\frac{\partial s}{\partial W^{(1)}} = \lambda W^{(1)}$，$\frac{\partial s}{\partial W^{(2)}} = \lambda W^{(2)}$。因为$\vec{o} = W^{(2)} \vec{h}$，所以$\frac{\partial \vec{o}}{\partial (W^{(2)})^\top} = \vec{h}$。因此<font color='blue'>$\frac{\partial J}{\partial W^{(2)}} = prod(\frac{\partial J}{\partial \vec{o}}, \frac{\partial \vec{o}}{\partial W^{(2)}}) + prod(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial W^{(2)}}) = prod(\frac{\partial L}{\partial \vec{o}}, \vec{h}) + \lambda W^{(2)}$</font>。到目前为止，已经算出了输出层权重的梯度。

**计算$\frac{\partial J}{\partial W^{(1)}}$**:

因为$\frac{\partial \vec{o}}{\partial \vec{h}} = (W^{(2)})^\top$，所以<font color='blue'>$\frac{\partial J}{\partial \vec{h}} = prod(\frac{\partial L}{\partial \vec{o}}, (W^{(2)})^\top)$</font>；

进一步地，<font color='blue'>$\frac{\partial J}{\partial \vec{z}} = prod(\frac{\partial J}{\partial h}, \frac{\partial h}{\partial z}) = prod\big(\frac{\partial L}{\partial \vec{o}}, (W^{(2)})^\top\big) \odot \phi'(\vec{z})$</font>；

最终，<font color='blue'>$\frac{\partial J}{\partial W^{(1)}} = prod(\frac{\partial J}{\partial \vec{z}}, \frac{\partial \vec{z}}{\partial W^{(1)}}) + prod(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial W^{(1)}}) = prod\Big(prod\big(\frac{\partial L}{\partial \vec{o}}, (W^{(2)})^\top\big) \odot \phi'(\vec{z}), \vec{x}\Big) + \lambda W^{(1)}$</font>。

在模型参数初始化完成后，我们交替地进行正向传播和反向传播，并根据反向传播计算的梯度迭代模型参数。**我们在反向传播中使用了正向传播中计算得到的中间变量来避免重复计算，这导致正向传播结束后不能立即释放中间变量内存，因此训练要比预测占用更多的内存。**另外需要指出的是，这些中间变量的个数大体上与网络层数线性相关，每个变量的大小跟批量大小和输入个数也是线性相关的，它们是导致较深的神经网络使用较大批量训练时更容易超内存的主要原因。

### 6 卷积神经网络
#### 6.1 二维互相关运算
互相关运算如图所示（$互相关 = rot180 (卷积)$）：
<img src='../resource/correlation.svg'>

#### 6.2 填充与步长
对于单个维度而言，设卷积核大小为$m$，步长为$s$，输入神经元两端各补$p$个零，则输出神经元的数量为
$$
\lfloor \frac{n - m + 2p}{s} \rfloor + 1.
$$
* 窄卷积：$s=1, p=0 \rightarrow n - m + 1$
* 宽卷积：$s=1, p=m-1 \rightarrow n + m - 1$
* 等宽卷积：$s=1,p=\frac{m-1}{2} \rightarrow n$

#### 6.3 多输入通道与多输出通道
若输入数据的通道数为$c_i$，则卷积核应当是一个大小为$c_i \times k_h \times k_w$的tensor。
由于输入和卷积核各有$c_i$个通道，我们可以在各个通道上对输入的二维数组和卷积核的二维核数组做互相关运算，再将这$c_i$个互相关运算的二维输出按通道相加，得到一个二维数组。这就是含多个通道的输入数据与多输入通道的卷积核做二维互相关运算的输出。
<img src='../resource/conv_multi_in.svg'>

#### 6.4 $1 \times 1$卷积层
输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1 \times 1$卷积层的作用与全连接层等价（区别在于$1 \times 1$卷积层允许权重参数共享，因而拥有更少的参数数量）。

$1×1$卷积层被当作保持高和宽维度形状不变的全连接层使用,可以通过调整网络层之间的通道数来控制模型复杂度。
<img src='../resource/conv_1x1.svg'>

#### 6.5 池化层
池化层可以缓解卷积层对位置的过度敏感性。

将卷积层的输出作为2×2最大池化的输入。设该卷积层输入是X、池化层输出为Y。无论是X[i, j]和X[i, j+1]值不同，还是X[i, j+1]和X[i, j+2]不同，池化层输出均有Y[i, j]=1。也就是说，使用2×2最大池化层时，只要卷积层识别的模式在高和宽上移动不超过一个元素，我们依然可以将它检测出来。
<img src='../resource/pooling.svg'>

#### 6.5 AlexNet
AlexNet包含8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。

AlexNet第一层中的卷积窗口形状是$11 \times 11$。因为ImageNet中绝大多数图像的高和宽均比MNIST图像的高和宽大10倍以上，ImageNet图像的物体占用更多的像素，所以需要更大的卷积窗口来捕获物体。第二层中的卷积窗口形状减小到$5×5$，之后全采用$3×3$。此外，第一、第二和第五个卷积层之后都使用了窗口形状为$3×3$、步幅为2的最大池化层。而且，AlexNet使用的卷积通道数也大于LeNet中的卷积通道数数十倍。

<img src='../resource/alexnet.png'>

#### 6.6 VGG
VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为$3×3$的卷积层后接上一个步幅为2、窗口形状为$2×2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。我们使用``vgg_block``函数来实现这个基础的VGG块，它可以指定卷积层的数量和输入输出通道数。

对于给定的感受野（与输出有关的输入图片的局部大小），**采用堆积的小卷积核优于采用大的卷积核，因为可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）**。例如，在VGG中，使用了3个3x3卷积核来代替7x7卷积核，使用了2个3x3卷积核来代替5*5卷积核，这样做的主要目的是在保证具有相同感知野的条件下，提升了网络的深度，在一定程度上提升了神经网络的效果。

#### 6.7 Network in Network (NiN)
卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维。可以用$1×1$卷积层代替全连接层，其中**空间维度（高和宽）上的每个元素相当于样本，通道相当于特征**。
<img src='../resource/nin.svg'>

#### 6.7 GoogLeNet
GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。

<img src='../resource/inception.svg'>

GoogLeNet中的基础卷积块叫作Inception块，有4条并行的线路。前3条线路使用窗口大小分别是1×1、3×3和5×5的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做1×1卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用3×3最大池化层，后接1×1卷积层来改变通道数。**4条线路都使用了合适的填充来使输入与输出的高和宽一致**。最后**将每条线路的输出在通道维上连结，并输入接下来的层中去**。

#### 6.8 批量归一化
通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。**批量归一化**和**残差网络**为训练和设计深度模型提供了两类重要思路。

**对全连接层批量归一化**:

使用批量归一化的全连接层的输出为
$$
\phi(BN(x)) = \phi (BN (Wu + b)),
$$
其中$u$为全连接层的输入，$BN$为批量归一化运算符。

对于小批量的**仿射变换的输出**$\mathcal{B} = \{ x^{(1)}, ..., x^{(m)} \}$，其中$x^{(i)} \in \mathbb{R}^d$，则批量归一化的输出为
$$
y^{(i)} = BN(x^{(i)}) \in \mathbb{R}^d.
$$

$BN(\cdot)$的具体步骤如下：

首先对小批量$\mathcal{B}$求均值和方差：
$$
\mu_{\mathcal{B}} \leftarrow \frac{1}{m} \sum_{i=1}^m x^{(i)},\\
\sigma^2_{\mathcal{B}} \leftarrow \frac{1}{m-1} \sum_{i=1}^m (x^{(i)} - \mu_{\mathcal{B}})^2.
$$

其中的平方计算是按元素求平方。接下来，使用按元素开方和按元素除法对$x^{(i)}$标准化：
$$
\hat{x}^{(i)} \leftarrow \frac{x^{(i)} - \mu_{\mathcal{B}}}{\sqrt{\sigma^2_{\mathcal{B}} + \epsilon}},
$$
这里$\epsilon > 0$是一个很小的常数，保证分母大于0。

批量归一化层引入了两个可以学习的模型参数，拉伸（scale）参数$\gamma$和偏移（shift）参数$\beta$。这两个参数和$x^{(i)}$形状相同，皆为$d$维向量。它们与分别做按元素乘法和加法计算：
$$
y^{(i)} \leftarrow \gamma \odot \hat{x}^{(i)} + \beta.
$$

可学习的拉伸和偏移参数保留了不对$x^{(i)}$做批量归一化的可能：
此时只需学出$\gamma = \sqrt{\sigma^2_{\mathcal{B}} + \epsilon}$，$\beta = \mu_{\mathcal{B}}$。我们可以对此这样理解：如果批量归一化无益，理论上，学出的模型可以不使用批量归一化。

**对卷积层批量归一化**:

对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数，并均为**标量**。

设小批量中有$m$个样本。在单个通道上，假设卷积计算输出的高和宽分别为$p$和$q$。我们需要对该通道中$m×p×q$个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中$m×p×q$个元素的均值和方差。

**预测时的批量归一化**:

使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

#### 6.9 ResNet
普通的网络结构（左）与加入残差连接的网络结构（右）:
<img src='../resource/residual-block.svg'>
在右图所示的残差块中，虚线框内要学习的是残差映射$f(x) - x$，当理想映射接近恒等映射时（即$f(x) = x$），虚线框内上方的加权运算的权重和偏差参数会被学习为0。此时的残差映射可以捕捉恒等映射的细微波动。

ResNet第一层与GooLeNet第一层一样，在输出通道数为64、步幅为2的7×7卷积层后接步幅为2的3×3的最大池化层。不同之处在于ResNet在卷积层后增加的批量归一化层。

GoogLeNet在后面接了4个由Inception块组成的模块。ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块，第一个模块的通道数同输入通道数一致。每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

#### 6.10 DenseNet
DenseNet里模块B的输出不是像ResNet那样和模块A的输出相加，而是在通道维上连结。这样模块A的输出可以直接传入模块B后面的层。

<img src='../resource/densenet.svg'>

DenseNet的主要构建模块是稠密块（dense block）和过渡层（transition layer）。前者定义了输入和输出是如何连结的，后者则用来控制通道数，使之不过大。